# 🤖 워크플로우에서 에이전트 사용하기

> **📖 Learn 사이트 링크**  
> https://learn.microsoft.com/en-us/agent-framework/tutorials/workflows/agents-in-workflows?pivots=programming-language-python  
>
> **💻 원본 소스 코드**  
> https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/workflows/agents/azure_ai_agents_streaming.py

## 📋 개요

이 과정에서는 Agent Framework를 사용하여 **AI 에이전트를 워크플로우에 통합**하는 방법을 보여줍니다. 

콘텐츠 생성, 검토 및 기타 협업 작업을 위해 특화된 AI 에이전트의 기능을 활용하는 워크플로우를 만드는 방법을 다룹니다.

### 🎯 학습 목표

다음과 같은 워크플로우를 생성합니다:

| 순서 | 단계 | 설명 |
|------|------|------|
| 1️⃣ | **에이전트 생성** | Azure AI Agent Service를 사용하여 지능형 에이전트를 생성 |
| 2️⃣ | **Writer 에이전트** | 프롬프트에 따라 콘텐츠를 생성하는 에이전트 구현 ✍️ |
| 3️⃣ | **Reviewer 에이전트** | 콘텐츠에 대한 피드백을 제공하는 에이전트 구현 👀 |
| 4️⃣ | **워크플로우 연결** | 에이전트들을 순차적인 파이프라인으로 연결 🔗 |
| 5️⃣ | **실시간 스트리밍** | 에이전트들이 요청을 처리하는 동안 실시간 업데이트 스트리밍 📡 |

## 📦 1단계: 필수 종속성 가져오기

먼저 Azure AI 에이전트 및 워크플로에 필요한 구성 요소를 가져옵니다.

In [14]:
import asyncio
import os

from agent_framework import AgentResponseUpdate, WorkflowBuilder
from agent_framework.azure import AzureOpenAIResponsesClient, AzureOpenAIChatClient
from azure.identity import AzureCliCredential

## 🏭 2단계: 공유 클라이언트 생성하기

멀티 에이전트를 생성하는 데 사용할 공유 LLM 클라이언트를 만듭니다.

In [15]:
# 여러 에이전트에서 사용할 공유 클라이언트 생성
client = AzureOpenAIChatClient(
    credential=AzureCliCredential(),
)

## 🤖 3단계: Agent 생성하기

컨텐츠 생성과 리뷰를 담당하는 **2개의 특화된 에이전트**를 생성합니다.

### 에이전트 역할

| 에이전트 | 역할 | 아이콘 |
|----------|------|--------|
| **Writer** | 훌륭한 콘텐츠 작성 및 편집 | ✍️📝 |
| **Reviewer** | 실행 가능한 피드백 제공 | 👀✅ |

In [16]:
# 컨텐츠를 생성하는 Writer 에이전트 생성
writer = client.as_agent(
    name="Writer",
    instructions=(
        "당신은 훌륭한 콘텐츠 작성자입니다. "
        "새로운 콘텐츠를 만들고 피드백에 따라 콘텐츠를 편집합니다."
        "한국어로 콘텐츠를 제공해주세요. ✍️📝"
    ),
)

# 피드백을 제공하는 Reviewer 에이전트 생성
reviewer = client.as_agent(
    name="Reviewer",
    instructions=(
        "당신은 훌륭한 콘텐츠 리뷰어입니다. "
        "작성자에게 제공된 콘텐츠에 대한 실행 가능한 피드백을 제공하세요. "
        "가능한 한 간결하게 피드백을 제공하세요. "
        "한국어로 피드백을 제공해주세요. 👀✅"
    ),
)

## 🔄 4단계: 워크플로우를 구성하고, 스트리밍을 실행합니다

빌더를 사용하여 에이전트를 **순차적인 워크플로우**에 연결하고, 에이전트들로부터 실시간 업데이트를 관찰하기 위해 스트리밍으로 워크플로우를 실행합니다.

In [21]:
# 에이전트를 executor로 사용하여 워크플로우를 구축합니다.
workflow = (
    WorkflowBuilder(start_executor=writer)
    .add_edge(writer, reviewer)
    .build()
)

last_executor_id: str | None = None

events = workflow.run("가성비 있고 쾌적한 전기 SUV를 위한 슬로건을 만드세요", stream=True)
async for event in events:
    if event.type == "output" and isinstance(event.data, AgentResponseUpdate):
        # 에이전트로부터 스트리밍 업데이트를 처리합니다.
        eid = event.executor_id
        if eid != last_executor_id:
            if last_executor_id is not None:
                print()
            print(f"{eid}:", end="\n" + "-"*60 + "\n", flush=True)
            last_executor_id = eid
        print(event.data, end="", flush=True)
    elif event.type == "output":
        print("\n===== 최종 출력 =====")
        print(event.data)

Writer:
------------------------------------------------------------
물론입니다! 가성비 있고 쾌적한 전기 SUV를 위한 슬로건을 여러 가지로 제안드립니다.

1. "합리적인 선택, 여유로운 드라이브 – 가성비 전기 SUV"
2. "가성비와 쾌적함, 둘 다 잡았다!"
3. "현명한 전기, 쾌적한 여정"
4. "지갑은 가볍게, 경험은 넓게 – 전기 SUV의 새 기준"
5. "경제성과 안락함, 전기 SUV로 한 번에"
6. "현명하게 타고, 쾌적하게 즐기다"
7. "최고의 효율, 최고의 안락함"
8. "효율의 가치, 쾌적의 경험 – 전기 SUV"
9. "가성비로 만나는 프리미엄 전기 SUV"
10. "합리적 가격, 프리미엄 라이프"

필요에 따라 한 문장 스타일 또는 살짝 변형도 도와드릴 수 있습니다! 더 원하시는 스타일이나 톤이 있다면 말씀해 주세요.
Reviewer:
------------------------------------------------------------
장점: 여러 슬로건을 다양하게 제시하여 선택의 폭을 넓혀주셨습니다. 간결하고 SUV의 특징(가성비, 쾌적함)이 잘 나타나 있습니다.

보완점: 제안한 슬로건 중 일부는 다소 일반적이거나 기존에 많이 쓰이는 표현과 유사합니다. 이런 부분은 브랜드만의 차별화된 메시지나 창의적인 언어를 추가하면 더욱 돋보일 수 있습니다. 예를 들어, SUV의 강인함, 친환경 전기차의 미래지향성, 가족 또는 라이프스타일 연계 등 추가적인 콘셉트를 녹여보면 좋겠습니다. 

총평: 목적에 부합하는 슬로건을 잘 모았습니다. 차별화된 포인트를 반영한 2~3개의 대표 슬로건을 더욱 다듬어보길 권장합니다.

### (옵션) 에이전트 Executor 간에 세션을 공유

기본적으로 각 에이전트 실행기는 자체 세션 상태를 사용합니다. 하지만, 동일한 AzureOpenAIResponsesClient에서 생성된 에이전트의 경우에는 공유 세션을 명시적으로 연결할 수 있습니다.

```python
    # 공유 세션을 생성하고, Executor에 지정합니다.
    shared_session = writer.create_session()
    writer_executor = AgentExecutor(writer, session=shared_session)
    reviewer_executor = AgentExecutor(reviewer, session=shared_session)
```

In [ ]:
# 예시 코드 (실행하지 마세요)
from agent_framework import InMemoryHistoryProvider, AgentExecutor

writer = client.as_agent(
    instructions=(
        "당신은 훌륭한 콘텐츠 작성자입니다. "
        "새로운 콘텐츠를 만들고 피드백에 따라 콘텐츠를 편집합니다."
    ),
    name="writer",
    context_providers=[InMemoryHistoryProvider()],
    # 에이전트에 동일한 context provider(예, InMemoryHistoryProvider)와 세션을 설정해야 합니다.
)

reviewer = client.as_agent(
    instructions=(
        "당신은 훌륭한 콘텐츠 리뷰어입니다. "
        "작성자에게 제공된 콘텐츠에 대한 실행 가능한 피드백을 제공하세요. "
        "가능한 한 간결하게 피드백을 제공하세요. "
    ),
    name="reviewer",
    context_providers=[InMemoryHistoryProvider()],
    # 에이전트에 동일한 context provider(예, InMemoryHistoryProvider)와 세션을 설정해야 합니다.
)

# 공유 세션을 생성하고 Executor에 지정합니다.
shared_session = writer.create_session()
writer_executor = AgentExecutor(writer, session=shared_session)
reviewer_executor = AgentExecutor(reviewer, session=shared_session)

# executor로 사용하여 워크플로우를 구축합니다.
workflow = (
    WorkflowBuilder(start_executor=writer_executor)
    .add_edge(writer_executor, reviewer_executor)
    .build()
)

last_executor_id: str | None = None

events = workflow.run("가성비 있고 쾌적한 전기 SUV를 위한 슬로건을 만드세요", stream=True)
async for event in events:
    if event.type == "output" and isinstance(event.data, AgentResponseUpdate):
        # 에이전트로부터 스트리밍 업데이트를 처리합니다.
        eid = event.executor_id
        if eid != last_executor_id:
            if last_executor_id is not None:
                print()
            print(f"{eid}:", end="\n" + "-"*60 + "\n", flush=True)
            last_executor_id = eid
        print(event.data, end="", flush=True)
    elif event.type == "output":
        print("\n===== 최종 출력 =====")
        print(event.data)

writer:
------------------------------------------------------------
물론입니다! 가성비 있고 쾌적한 전기 SUV를 위한 슬로건을 여러 가지 제안드립니다.

1. "실속과 편안함, 전기로 달리다"
2. "가성비 UP, 쾌적함 MAX! 전기 SUV의 새로운 기준"
3. "합리적인 선택, 프리미엄 전기 SUV"
4. "합리적인 전기, 쾌적한 주행"
5. "경제성에 안락함을 더하다, 전기 SUV"
6. "스마트한 가격, 완벽한 드라이브"
7. "지갑은 가볍게, 주행은 쾌적하게!"
8. "전기 SUV, 가성비의 혁신을 경험하다"
9. "합리적으로 즐기는 프리미엄 SUV 라이프"
10. "가성비 좋은 전기SUV, 쾌적함은 기본!"

특정 브랜드의 성격이나 대상 소비층을 조금 더 알려주신다면 더욱 맞춤화된 슬로건도 제작해 드릴 수 있습니다.
reviewer:
------------------------------------------------------------
슬로건이 중복 제안 형식이라 메시지가 강하게 전달되지 않습니다. 한두 개의 간결하고 임팩트 있는 슬로건으로 정리해 주세요. 예를 들어,  
"가성비로 만나는 프리미엄 전기 SUV, 쾌적함은 기본"  
처럼 브랜드의 핵심 가치를 명확히 드러내는 문구를 사용하면 좋습니다.  
또한, SUV의 장점(공간, 안전성 등)을 슬로건에 반영하면 더욱 효과적입니다.

---

## ⚙️ 동작 방식

| 단계 | 설명 |
|------|------|
| 🔐 **클라이언트 설정** | Azure CLI 자격 증명으로 `AzureOpenAIChatClient`를 구성하여 인증합니다 |
| 🤖 **에이전트 생성** | 동일한 LLM 클라이언트를 공유하여 Writer 및 Reviewer 에이전트를 만듭니다 |
| 🔗 **순차 처리** | Writer가 먼저 콘텐츠를 생성한 후 Reviewer에게 전달합니다 |
| 📡 **스트리밍 업데이트** | `type="output"` 이벤트와 `AgentResponseUpdate`를 통해 실시간 토큰을 수신합니다 |
| 🧵 **공유 세션** *(선택 사항)* | 동일한 클라이언트를 사용하여 에이전트 간 공유 메모리/스레드를 구성할 수 있습니다 |

---

## 💡 핵심 개념

> **`AzureOpenAIChatClient`**  
> 일관된 구성으로 워크플로우 에이전트를 생성하는 데 사용되는 공유 클라이언트입니다.

> **`WorkflowEvent`**  
> `type="output"` 이벤트에는 에이전트 출력 데이터가 포함됩니다.  
> 스트리밍 → `AgentResponseUpdate` / 비스트리밍 → `AgentResponse`

> **Sequential Workflow**  
> 출력이 한 에이전트에서 다음 에이전트로 흐르는 파이프라인입니다.

> **Shared Session Pattern**  
> 워크플로우에서 선택된 에이전트 간 공유 메모리/스레드를 구성하는 선택적 패턴입니다.

---

## 📚 완전한 구현 샘플

| 샘플 | 링크 |
|------|------|
| 🌊 스트리밍 | [azure_ai_agents_streaming.py](https://github.com/microsoft/agent-framework/blob/main/python/samples/03-workflows/agents/azure_ai_agents_streaming.py) |
| 🧵 공유 세션 | [azure_ai_agents_with_shared_session.py](https://github.com/microsoft/agent-framework/blob/main/python/samples/03-workflows/agents/azure_ai_agents_with_shared_session.py) |
